In [ ]:
# Zaimplementować algorytm klasyfikacji binarnej kNN (k najbliższych sąsiadów).
# Należy udostępnić metody train i predict.
# Train buduje bazę przypadków uczących (przyjmuje przynajmniej wektory i prawidłowe odpowiedzi).
# Wielokrotne wywołanie metody train powinno rozszerzać zbiór przypadków uczących.
# Metoda predict przyjmuje wektor (opcjonalnie: większą liczbę wektorów naraz) i zwraca odpowiedź klasyfikatora.
# Należy umożliwić wybór jednej z czterech funkcji odległości: euklidesowej, taksówkowej, maksimum i cosinusowej.

# Porównać wyniki na podanych zbiorach dla przynajmniej 3 wartości k (ostatnia kolumna zawiera etykietę).

# Pomocna może być metoda np.array.partition.
# Należy użyć biblioteki numpy.

# Termin: wt. 17.12.2024 08:00

import csv
import numpy as np

class KNNClassifier:
    def __init__(self, distance_metric = 'euclidean'): # enum lepszy niż string, przekazywać funkcje jako parametr
        
        valid_metrics = ['euclidean', 'manhattan', 'chebyshev', 'cosine'] # lepiej zbiór bo lepsza optymalizacja
        if distance_metric not in valid_metrics:
            raise ValueError(f"Unsupported distance metric: {distance_metric}. Valid options are: {valid_metrics}")
        
        self.distance_metric = distance_metric
        self.x_train = None
        self.y_train = None

    def train(self, x, y):
        if self.x_train is None and self.y_train is None:
            self.x_train = x
            self.y_train = y
        else:
            if x.ndim == 1 :
                try:
                    x = x.reshape(-1, self.x_train.shape[1])
                except ValueError:
                    raise ValueError(f"Cannot reshape array {x} into shape {self.x_train.shape[1]}")

            self.x_train = np.vstack([self.x_train, x])
            self.y_train = np.hstack([self.y_train, y])

    def predict(self, x, k = 3):
        if x.shape[0] != self.x_train.shape[1]:
            raise ValueError(f"Input vector dimension {x.shape[0]} does not match training data dimension {self.x_train.shape[1]}")

        if self.distance_metric == 'euclidean':
            distances = []
            for i in range(len(self.x_train)):
                suma = 0
                for j in range(len(self.x_train[i])):
                    suma += ( x[j] - self.x_train[i][j] )**2
                distances.append( [round( suma**0.5, 3 ), self.y_train[i]] )
            #((x-self.x_train)**2).sum(axis=1)**0.5

        elif self.distance_metric == 'manhattan':
            distances = []
            for i in range(len(self.x_train)):
                distance = 0
                for j in range(len(self.x_train[i])):
                    distance += abs( x[j] - self.x_train[i][j] )
                distances.append( [distance, self.y_train[i]] )

        elif self.distance_metric == 'chebyshev':
            distances = []
            for i in range(len(self.x_train)):
                distance = []
                for j in range(len(self.x_train[i])):
                    distance.append( abs( x[j] - self.x_train[i][j] ) )
                distances.append( [max(distance), self.y_train[i]] )
        
        elif self.distance_metric == 'cosine':
            min_val = np.min(self.x_train)
            max_val = np.max(self.x_train)
            if min_val < 0:
                x = (x - min_val) / (max_val - min_val)
                self.x_train = (self.x_train - min_val) / (max_val - min_val)
  
            distances = []
            norm_x = np.linalg.norm(x)
            for i in range(len(self.x_train)):
                numerator = np.dot(x, self.x_train[i])
                norm_x_train = np.linalg.norm(self.x_train[i])
                denominator = norm_x * norm_x_train
                distances.append( [1- numerator / denominator, self.y_train[i]])

        distances = np.array(distances)
        k_nearest_distances = distances[np.argpartition(distances[:, 0], k)[:k]]
        labels = k_nearest_distances[:, 1].astype(int)
        prediction = np.argmax(np.bincount(labels))
        return prediction

def open_csv(file_path):
    x, y = [], []
    with open(file_path, 'r') as csvfile:
        csvreader = csv.reader(csvfile, delimiter=' ')
        for row in csvreader:
            x.append(list(map(float, row[:-1])))
            y.append(int(row[-1]))
    return np.array(x), np.array(y) 

#############################################################################################################################################

x, y = open_csv('C:/moje pliki/dokumenty i pliki/studia/magisterka_Uczenie_Maszynowe/Jezyk Python/jezyk_python_lab/dataset0.csv')       
print(x)
print(y)
# Testowanie klasyfikatora
knn = KNNClassifier(distance_metric='euclidean')
knn.train(x, y)
prediction = knn.predict(np.array([1.0, 2.0]))
print(f'Prediction: {prediction}') # 1

knn.train(np.array([1., 2.]), np.array(1))
prediction = knn.predict(np.array([14.0, 23.0]))
print(f'Prediction: {prediction}') # 1

#############################################################################################################################################

x, y = open_csv('C:/moje pliki/dokumenty i pliki/studia/magisterka_Uczenie_Maszynowe/Jezyk Python/jezyk_python_lab/dataset1.csv')
# Testowanie klasyfikatora
knn = KNNClassifier(distance_metric='chebyshev')
knn.train(x, y)
prediction = knn.predict(np.array([121.314024, 222.530757]), k = 5)
print(f'Prediction: {prediction}') # 1

knn.train(np.array([121.314024, 222.530757]), np.array(1))
prediction = knn.predict(np.array([0.45540533, 6.420678]))
print(f'Prediction: {prediction}') # 0

#############################################################################################################################################

x, y = open_csv('C:/moje pliki/dokumenty i pliki/studia/magisterka_Uczenie_Maszynowe/Jezyk Python/jezyk_python_lab/dataset2.csv')
# Testowanie klasyfikatora
knn = KNNClassifier(distance_metric='manhattan')
knn.train(x, y)
prediction = knn.predict(np.array([121.314024, 222.530757, 23.3578, 87.4542]))
print(f'Prediction: {prediction}') # 1

knn.train(np.array([121.314024, 222.530757, 23.3578, 87.4542]), np.array(1))
prediction = knn.predict(np.array([0.45540533, 6.420678, 86.3435, 2.3546]))
print(f'Prediction: {prediction}') # 0

x, y = open_csv('C:/moje pliki/dokumenty i pliki/studia/magisterka_Uczenie_Maszynowe/Jezyk Python/jezyk_python_lab/dataset2.csv')
# Testowanie klasyfikatora
knn = KNNClassifier(distance_metric='euclidean')
knn.train(x, y)
prediction = knn.predict(np.array([121.314024, 222.530757, 23.3578, 87.4542]))
print(f'Prediction: {prediction}') # 1

knn.train(np.array([121.314024, 222.530757, 23.3578, 87.4542]), np.array(1))
prediction = knn.predict(np.array([0.45540533, 6.420678, 86.3435, 2.3546]))
print(f'Prediction: {prediction}') # 0

#############################################################################################################################################
# Sprawdzenie wyłapywania wyjątków

x, y = open_csv('C:/moje pliki/dokumenty i pliki/studia/magisterka_Uczenie_Maszynowe/Jezyk Python/jezyk_python_lab/dataset0.csv')       
knn = KNNClassifier(distance_metric='eucidean')
knn.train(x, y)
prediction = knn.predict(np.array([1.0, 2.0]))
print(f'Prediction: {prediction}')

x, y = open_csv('C:/moje pliki/dokumenty i pliki/studia/magisterka_Uczenie_Maszynowe/Jezyk Python/jezyk_python_lab/dataset0.csv')       
knn = KNNClassifier(distance_metric='eucidean')
knn.train(x, y)
knn.train(np.array([1., 2., 3., 4.]), np.array(1))
prediction = knn.predict(np.array([14.0, 23.0, 2.0]))
print(f'Prediction: {prediction}')